## set up

In [1089]:
# imports
import os
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
import json
from pydantic import BaseModel, Field
import string
import re
from typing import Optional 
import asyncio
import aiofiles

In [1315]:
# Define a semaphore to limit concurrency
SEMAPHORE_LIMIT = 20
semaphore = asyncio.Semaphore(SEMAPHORE_LIMIT)

async def semaphore_wrapper(func, *args):
    """Wrap async function with semaphore to limit concurrency"""
    async with semaphore:
        return await func(*args)

In [2]:
# api keys
try:
    with open("../secrets.json") as f:
        secrets = json.load(f)
    open_ai_key = secrets["openai"]
    anthropic_key = secrets["anthropic"]
    google_key = secrets["google"]
    os.environ["OPENAI_API_KEY"] = open_ai_key
    os.environ["ANTHROPIC_API_KEY"] = anthropic_key
    os.environ["GOOGLE_API_KEY"] = google_key
    print("API key loaded.")
except FileNotFoundError:
    print("Secrets file not found. YOU NEED THEM TO RUN THIS.")

API key loaded.


In [3]:
## make sure all folders are created inside notebooks folder
if not os.path.exists("scripts"):
    os.makedirs("scripts")
if not os.path.exists("json"):
    os.makedirs("json")
if not os.path.exists("chapters"):
    os.makedirs("chapters")

In [4]:
# NOT VERY RELEVANT
def format_gemini_response(res):
    output_formatted = res.content[7:-3]
    # turn into json
    output = json.loads(output_formatted)
    return output

class DotDict:
    # allow dot notation for dicts
    def __init__(self, dictionary):
        # check if dictionary is a string
        if isinstance(dictionary, str):
            # if it is a string, convert it to a dictionary
            print(dictionary)
            dictionary = json.loads(dictionary)
        for key, value in dictionary.items():
            setattr(self, key, value)

In [5]:
model_type = "default"
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
# llm = ChatOpenAI(model="gpt-4o", temperature=0.1)
llm2 = ChatAnthropic(model_name="claude-3-5-sonnet-20240620", temperature=0.1, max_tokens_to_sample=4096, api_key=os.getenv("ANTHROPIC_API_KEY"))

# NOT USING RIGHT NOW
# model_type = "gemini"
llm3 = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.1)

# safety_settings={
#     HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
# }

# # original method
# og_generate = ChatGoogleGenerativeAI._generate

# # patch

# ChatGoogleGenerativeAI._generate = partial(llm._generate, safety_settings=safety_settings)

In [6]:
# Pydantic
class StoryInfo(BaseModel):
    """Get overall information about the story."""
    numChapters: int = Field(description="Number of chapters in the story")
    firstline: str = Field(description="First line of the story")
    lastline: Optional[str] = Field(description="Last line of the story or first volume")
    markers: list[str] = Field(description="List of markers to split the story into chapters, as formatted in the table of contents. Make sure the formatting matches the markers in the actual text (including the word 'chapter' if applicable). Keep all punctuation marks and capitalization, and use the full chapter names. If there are multiple volumes, focus on the first volume.")

def get_marker_prompt(story):
    prompt = f'''List the markers needed to split this story into chapters: {story}. Format your response like Story Info:
    class StoryInfo(BaseModel):
        """Get overall information about the story."""
        firstline: str = Field(description="First line of the story")
        lastline: Optional[str] = Field(description="Last line of the story or first volume")
        markers: list[str] = Field(description="List of markers to split the story into chapters. Keep all punctuation marks and capitalization.")
    '''
    return prompt

# structured_llm = llm2.with_structured_output(StoryInfo)
structured_llm = llm.with_structured_output(StoryInfo)

## split text into chapters

In [2498]:
# NOTE: Make sure your story file is in the scripts folder (as a txt file)
# read in txt file from scripts folder (inside notebooks folder)
og_story_name = "yourname" # name of the story (should match the name of the file (e.g., type "color" for color.txt)
story_name = og_story_name
analysis_type = "theme" # character, theme

In [2499]:
# optional, but this part needs to be manually looked up
story_info = {
    "title": "Your Name",
    "type": "Movie",
    "director": "Makoto Shinkai",
    "year": "2016",
    "url": "https://archive.org/stream/manga_Kimi_no_Na_wa/Kimi%20no%20Na%20wa%20%5BYen%20Press%5D_djvu.txt",
    "image": "https://m.media-amazon.com/images/M/MV5BODRmZDVmNzUtZDA4ZC00NjhkLWI2M2UtN2M0ZDIzNDcxYThjL2ltYWdlXkEyXkFqcGdeQXVyNTk0MzMzODA@._V1_FMjpg_UX1000_.jpg",
}

In [2500]:
# read in the script
with open(f"scripts/{story_name}.txt") as f:
    story = f.read()

In [2501]:
if analysis_type == "theme":
    story_name += "_themes"
elif analysis_type == "2":
    story_name += "_2"

In [2373]:
if not os.path.exists(f"json/{story_name}"):
    os.makedirs(f"json/{story_name}")

In [1457]:
# focus on the first volume if there are multiple volumes
if "END OF VOL." in story:
    volumes = story.split("END OF VOL.")
    story = volumes[0]
elif "VOLUME II." in story:
    volumes = story.split("VOLUME II.")
    story = volumes[0]

In [1458]:
# split story into chunks
char_chunk = 120000
story_len = len(story)
print(story_len)
if story_len > char_chunk:
    # story_chunks = [story[i:i + char_chunk] for i in range(0, len(story), char_chunk)]
    # take 0:char_chunk/2 and -char_chunk/2: for the first chunk
    chunk = story[0:char_chunk//2] + story[-char_chunk//2:]
    story_chunks = [chunk]
else: # if story is less than 120k characters, use the whole story
    story_chunks = [story]
print(len(story_chunks))

484900
1


In [ ]:
all_res = []
for i, chunk in enumerate(story_chunks):
    invalid_input = True
    while invalid_input: # try until valid input
        # see if exception is thrown
        try:
            # if model_type == 'gemini':
            #     prompt = get_marker_prompt(story)
            #     markers = llm3.invoke(prompt)
            #     res = format_gemini_response(markers)
            #     print(res)
            #     res = DotDict(res['Story Info'])
            # else:
            res = structured_llm.invoke(chunk)
            all_res.append(res)
            print(f"Chunk {i}")
            invalid_input = False
        except Exception as e:
            print("Exception thrown. Trying again.")
            print("Error:", e)
            invalid_input = True
    # if i < len(story_chunks) - 1:
    #     # Pause for a specified time (e.g., 60 seconds) to avoid rate limits
    #     time.sleep(60)  # Adjust the sleep duration as per your rate limit requirements

In [109]:
# save first line from first chunk, last line from last chunk, and combine markers
first_line = all_res[0].firstline
last_line = all_res[-1].lastline
num_chapters = sum([res.numChapters for res in all_res])
markers = []
for res in all_res:
    markers.extend(res.markers)

num_chapters, markers

(12,
 ['CHAPTER I.     Down the Rabbit-Hole',
  'CHAPTER II.    The Pool of Tears',
  'CHAPTER III.   A Caucus-Race and a Long Tale',
  'CHAPTER IV.    The Rabbit Sends in a Little Bill',
  'CHAPTER V.     Advice from a Caterpillar',
  'CHAPTER VI.    Pig and Pepper',
  'CHAPTER VII.   A Mad Tea-Party',
  'CHAPTER VIII.  The Queen’s Croquet-Ground',
  'CHAPTER IX.    The Mock Turtle’s Story',
  'CHAPTER X.     The Lobster Quadrille',
  'CHAPTER XI.    Who Stole the Tarts?',
  'CHAPTER XII.   Alice’s Evidence'])

In [110]:
# first_line = res.firstline
# last_line = res.lastline
# markers = res.markers
first_line, last_line

('Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, "and what is the use of a book," thought Alice "without pictures or conversations?"',
 'and how she would gather about her other little children, and make _their_ eyes bright and eager with many a strange tale, perhaps even with the dream of Wonderland of long ago: and how she would feel with all their simple sorrows, and find a pleasure in all their simple joys, remembering her own child-life, and the happy summer days.')

In [2502]:
story = re.sub(r' {2,}', ' ', story)  # Replace consecutive spaces with one space
story = re.sub(r'\n{2,}', '\n', story)  # Replace consecutive newlines with one newline
story = re.sub(r' ?\n ?', '\n', story)  # Replace space + newline or newline + space with one newline
story = story.replace("“", "\"").replace("”", "\"").replace("‘", "'").replace("’", "'")

In [2503]:
story_formatted = story.replace("\n", " ")
# replace weird quotes too
story_formatted = story_formatted.replace("\"", " ").replace("'", " ")

In [432]:
# # remove any marker that doesn't include one of these words
# exclude_words = ["part", "preface", "contents", "by", "end", "epilogue", "letter"]
exclude_words = ["part", "preface", "contents", "introduction", "end", "epilogue", "letter", "volume"]
markers = [m for m in markers if not any(word in m.lower() for word in exclude_words)]

In [1466]:
# save as json 
story_json = {
    "first_line": first_line,
    "last_line": last_line,
    "markers": markers
}

if not os.path.exists(f"json/{og_story_name}"):
    os.makedirs(f"json/{og_story_name}")
with open(f"json/{og_story_name}/summary.json", "w") as f:
    json.dump(story_json, f, indent=4)

In [2440]:
# read in markers from summary.json
with open(f"json/{og_story_name}/summary.json") as f:
    summary = json.load(f)
    markers = summary["markers"]
    first_line = summary["first_line"]
    last_line = summary["last_line"]

In [2115]:
story_formatted[-100:] 

' help produce our new eBooks, and how to subscribe to our email newsletter to hear about new eBooks.'

In [435]:
first_ind = story_formatted.find(first_line.replace("\"", " ").replace("'", " "))
first_ind, first_line.replace("\"", " ").replace("'", " ")

(1430,
 'Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to unite some of the best blessings of existence; and had lived nearly twenty-one years in the world with very little to distress or vex her.')

In [29]:
second_marker = markers[1] + " "
last_line_index = story_formatted.find(second_marker, first_ind)
last_line_index, second_marker, story_formatted[last_line_index-50:last_line_index+50]

(34025,
 'II ',
 'd, and I was alone again in the unquiet darkness. II About halfway between West Egg and New York the')

In [25]:
last_ind = story_formatted.find(last_line)
last_ind, last_line

(268997,
 'So we beat on, boats against the current, borne back ceaselessly into the past.')

In [15]:
# split story into chapters based on markers
chapters = []
cur_first_line = first_line.replace("\"", " ").replace("'", " ")
cur_min_index = 0
for i, marker in enumerate(markers):
    # print("marker", marker)
    exists_next = i+1 < len(markers)
    next_marker = markers[i+1] if exists_next else ""
    # print("next_marker", next_marker)
    
    if exists_next and " ACT " in next_marker.upper():
        next_marker = next_marker.upper()
    formatted_last_line = next_marker if exists_next else last_line
    # print("formatted_last_line", formatted_last_line)

    if " ACT " not in next_marker.upper():
        formatted_last_line += " "
    cur_last_line = formatted_last_line
    cur_last_line = cur_last_line.replace("\"", " ").replace("'", " ")
    first_line_index = story_formatted.rfind(cur_first_line, cur_min_index)
    print("cur_first_line", repr(cur_first_line))
    print("first_line_index", first_line_index)
    print("Snippet around first_line_index:", repr(story_formatted[first_line_index:first_line_index + 200]))

    if i != 0 and " ACT " not in marker.upper():
        first_line_index += len(cur_first_line)
    last_line_index = story_formatted.find(cur_last_line, first_line_index)
    print("cur_last_line", repr(cur_last_line))
    print("last_line_index", last_line_index)
    print("Snippet around last_line_index:", repr(story_formatted[last_line_index-200:last_line_index]))
    
    if i+1 == len(markers):
        if last_line_index == -1:
            last_line_index = len(story_formatted)
        else:
            last_line_index += len(cur_last_line) 
    chapter = story[first_line_index:last_line_index].strip()

    # add line number in front of each line
    chapter = chapter.split("\n")
    for j, line in enumerate(chapter):
        chapter[j] = f"LINE {j+1}: {line}"
    chapter = "\n".join(chapter)
    chapters.append(chapter)
    cur_first_line = cur_last_line
    cur_min_index = last_line_index

cur_first_line 'It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.'
first_line_index 1858
Snippet around first_line_index: 'It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife. However little known the feelings or views of such a man may be on his first enter'
cur_last_line 'CHAPTER II. '
last_line_index 6309
Snippet around last_line_index: 'standing, little information, and uncertain temper. When she was discontented she fancied herself nervous. The business of her life was to get her daughters married; its solace was visiting and news. '
cur_first_line 'CHAPTER II. '
first_line_index 452883
Snippet around first_line_index: 'CHAPTER II. Elizabeth had settled it that Mr. Darcy would bring his sister to visit her, the very day after her reaching Pemberley; and was consequently resolved not to be out of sight of the inn the '
cur_last_line 'CHAPTER III. '
la

In [1326]:
# check results
for i, marker in enumerate(markers):
    print(f"Chapter {marker}")
    chapter_first_line = chapters[i][:100]
    chapter_last_line = chapters[i][-100:]
    print("first_line:", chapter_first_line)
    print("last_line:", chapter_last_line)
    print("\n")

Chapter Chapter I.
first_line: LINE 1: My father's family name being Pirrip, and my Christian name Philip, my
LINE 2: infant tongue
last_line:  could see no signs of
LINE 158: him. But now I was frightened again, and ran home without stopping.


Chapter Chapter II.
first_line: LINE 1: My sister, Mrs. Joe Gargery, was more than twenty years older than I,
LINE 2: and had establ
last_line: INE 292: I had entered when I ran home last night, shut it, and ran for the
LINE 293: misty marshes.


Chapter Chapter III.
first_line: LINE 1: It was a rimy morning, and very damp. I had seen the damp lying on the
LINE 2: outside of my
last_line: t
LINE 171: I heard of him, I stopped in the mist to listen, and the file was still
LINE 172: going.


Chapter Chapter IV.
first_line: LINE 1: I fully expected to find a Constable in the kitchen, waiting to take me
LINE 2: up. But not 
last_line: e of whom held out a
LINE 290: pair of handcuffs to me, saying, "Here you are, look sharp, come on!"


Chapter Chap

In [1534]:
# save to new txt files
# create folder for story if it doesn't exist
if not os.path.exists(f"chapters/{og_story_name}"):
    os.makedirs(f"chapters/{og_story_name}")
for i, marker in enumerate(markers):
    with open(f"chapters/{og_story_name}/{marker}.txt", "w") as f:
        f.write(chapters[i])
        print(f"Chapter {marker} saved.")

Chapter Prologue saved.
Chapter Act I saved.
Chapter Act II saved.
Chapter Act III saved.
Chapter Act IV saved.
Chapter Act V saved.


In [2504]:
# read in chapters from txt files
chapters = []
for i, marker in enumerate(markers):
    with open(f"chapters/{og_story_name}/{marker}.txt") as f:
        chapter = f.read()
        chapters.append(chapter)

In [2505]:
chapters[0]

'LINE 1: Chapter Two\nLINE 2: The Beginning\nLINE 3: I don\'t recognize that ringtone , I think drowsily.\nLINE 4: An alarm? But I\'m still sleepy. I was in the zone drawing last night and\nLINE 5: didn\'t get to bed until it was almost dawn.\nLINE 6: "...ki... Taki."\nLINE 7: Now somebody\'s calling my name. It\'s a girl\'s voice.A girl?\nLINE 8: "Taki, Taki."\nLINE 9: Her voice is earnest, pleading, as if she\'s about to cry. A voice trembling\nLINE 10: with loneliness, like the glimmer of distant stars.\nLINE 11: "Don\'t you remember me?" the voice asks me anxiously.\nLINE 12: No, I don\'t know you.\nLINE 13: Suddenly, the train stops, and the doors open. Oh, right—I was on a train.\nLINE 14: The moment I realize this, I\'m standing in a packed train car. A pair of wide\nLINE 15: eyes hovers right in front of me. A girl in a uniform is staring straight at me, but\nLINE 16: the press of disembarking passengers is pushing her farther away.\nLINE 17: "My name is Mitsuha!"\nLINE 18: The

## analyze scene

In [27]:
scene_type = "character" # location, character, or n/a

In [28]:
extra_scene_instruction = "Analyze the scenes in this chapter."
if scene_type == "location":
    extra_scene_instruction += " Start a new scene when the location changes."
elif scene_type == "character":
    extra_scene_instruction += " Start a new scene when new characters enter or old characters leave."

extra_scene_instruction += "\nChapter text:"

extra_scene_instruction

'Analyze the scenes in this chapter. Start a new scene when new characters enter or old characters leave.\nChapter text:'

In [29]:
# Pydantic
class SceneInfo(BaseModel):
    """Get information about a scene in the story."""
    scene: str = Field(description="Short title of scene (don't include chapter name or number). Each scene should have a unique title, and each title should be no longer than 6 words.")
    summary: str = Field(description="1 line summary of the scene")
    firstline: int = Field(description="First line number of the current scene in this chapter (should be after last line number of previous scene)")
    lastline: int = Field(description="Last line number of the current scene in this chapter (should be after first line number of current scene)")

class SceneListInfo(BaseModel):
    """Get overall information about the story."""
    scenelist: list[SceneInfo] = Field(description="List of key scenes in this chapter")

def get_scenelist_prompt(chapter):
    scene_prompt = f'''List the key scenes in this chapter: {chapter}. Format your response like SceneListInfo:
                    class SceneInfo(BaseModel):
                        """Get information about a scene in the story."""
                        scene: str = Field(description="Title of scene (don't include chapter name or number)")
                        summary: str = Field(description="1 line summary of the scene")
                        firstline: int = Field(description="First line number of the current scene in this chapter (should be after last line number of previous scene)")
                        lastline: int = Field(description="Last line number of the current scene in this chapter (should be after first line number of current scene)")

                    class SceneListInfo(BaseModel):
                        """Get overall information about the story."""
                        scenelist: list[SceneInfo] = Field(description="List of key scenes in this chapter")'''
    return scene_prompt

scene_llm = llm.with_structured_output(SceneListInfo)
# scene_llm = llm2.with_structured_output(SceneListInfo)

In [32]:
# Pydantic
class CharacterInfo(BaseModel):
    """Get information about a character in this scene."""
    name: str = Field(description="Full name of the character. Character name should be 5 words or less.")
    role: str = Field(description="Main actions and/or motivations of the character in this scene")
    importance: float = Field(description="Importance of the character in this scene from from 0: not important at all to 1: very important character")
    emotion: str = Field(description="Emotion of the character in this scene, described in a few words or a short phrase")
    sentiment: float = Field(description="Sentiment of the character in this scene from -1 (very negative) to 1 (very positive)")
    quote: str = Field(description="A direct quote from the character in this scene to illustrate their emotions and/or actions")
    fake_quote: str = Field(description="A brief explanation of the character's emotions and/or actions in this scene if you cannot find a direct quote in the text. If you can find a direct quote, leave this field empty.")

# Pydantic
class ThemeInfo(BaseModel):
    """Get information about a theme in this scene."""
    name: str = Field(description="Short general phrase to describe the theme. Theme name should be 3 words or less.")
    role: str = Field(description="Short description of the role of the theme in this scene")
    importance: float = Field(description="Importance of the theme in this scene from from 0: not important at all to 1: very important theme")
    emotion: str = Field(description="Emotion associated with the theme in this scene, described in a few words or a short phrase")
    sentiment: float = Field(description="Sentiment of the theme in this scene from -1 (very negative) to 1 (very positive)")
    quote: str = Field(description="A direct quote in this scene to illustrate the theme")
    fake_quote: str = Field(description="A brief explanation of the theme in this scene if you cannot find a direct quote in the text. If you can find a direct quote, leave this field empty.")

class SceneDetails(BaseModel):
    """Get overall information about the scene."""
    location: str = Field(description="Physical location of the scene. Location name should be 5 words or less.")
    importance: float = Field(description="Importance of the scene from 0: not important at all to 1: very important and pivotal scene")
    conflict: float = Field(description="Conflict in the scene from 0: no conflict to 1: high conflict")
    characters: list[CharacterInfo] = Field(description="List of key characters in the scene. Make sure there are no duplicates, and focus on speaking characters. Don't include any unnamed characters.")

class SceneDetailsTheme(BaseModel):
    """Get overall information about the scene."""
    location: str = Field(description="Physical location of the scene. Location name should be 5 words or less.")
    importance: float = Field(description="Importance of the scene from 0: not important at all to 1: very important and pivotal scene")
    conflict: float = Field(description="Conflict in the scene from 0: no conflict to 1: high conflict")
    themes: list[ThemeInfo] = Field(description="List of key themes in the scene. Make sure there are no duplicates but try to make the themes as general as possible.")

def get_scenedetails_prompt(scene):
    scene_prompt = f'''List the key details in this scene: {scene}. Format your response like SceneDetails:
                    class CharacterInfo(BaseModel):
                        """Get information about a character in this scene."""
                        name: str = Field(description="Full name of the character. Character name should be 5 words or less.")
                        role: str = Field(description="Main actions and/or motivations of the character in this scene")
                        importance: float = Field(description="Importance of the character in this scene from from 0: not important at all to 1: very important character")
                        emotion: str = Field(description="Emotion of the character in this scene, described in a few words or a short phrase")
                        sentiment: float = Field(description="Sentiment of the character in this scene from -1 (very negative) to 1 (very positive)")
                        quote: str = Field(description="A direct quote from the character in this scene to illustrate their emotions and/or actions")

                    class SceneDetails(BaseModel):
                        """Get overall information about the scene."""
                        location: str = Field(description="Physical location of the scene. Location name should be 5 words or less.")
                        importance: float = Field(description="Importance of the scene from 0: not important at all to 1: very important and pivotal scene")
                        conflict: float = Field(description="Conflict in the scene from 0: no conflict to 1: high conflict")
                        characters: list[CharacterInfo] = Field(description="List of key characters in the scene. Make sure there are no duplicates, and focus on speaking characters.  Don't include any unnamed characters.")'''
    return scene_prompt

scene_details_llm = llm.with_structured_output(SceneDetails)
scene_details_llm_theme = llm.with_structured_output(SceneDetailsTheme)
# scene_details_llm = llm2.with_structured_output(SceneDetails)

In [1064]:
async def get_scene_info_async(chapters, chapter_num):
    print(f"Splitting Chapter {markers[chapter_num]} into scenes...")

    cur_chapter = chapters[chapter_num]

    invalid_input = True
    while invalid_input: # try until valid input
        # see if exception is thrown
        try:
            if model_type == "gemini":
                prompt = get_scenelist_prompt(cur_chapter)
                chapter_scenes = await llm3.ainvoke(prompt)
                chapter_scenes = format_gemini_response(chapter_scenes)
            else:
                chapter_scenes = await scene_llm.ainvoke(extra_scene_instruction + "\n" + cur_chapter)
            invalid_input = False
        except:
            invalid_input = True
            
    if model_type == "gemini":
        chapter_scenes_list = [DotDict(scene) for scene in chapter_scenes['scenelist']]
    else: 
        chapter_scenes_list = chapter_scenes.scenelist
    formatted_chapter_scenes = []

    cur_last_line = 0

    for scene in chapter_scenes_list:
        scene_title = scene.scene
        scene_summary = scene.summary
        scene_first_line = scene.firstline
        scene_last_line = scene.lastline

        print(f"FIRST LINE: {scene_first_line}")
        print(f"LAST LINE: {scene_last_line}")
        print()

        formatted_scene = {
            "title": scene_title,
            "summary": scene_summary,
            "chapter": markers[chapter_num],
            "first_line": scene_first_line,
            "last_line": scene_last_line
        }

        if scene_last_line > scene_first_line and scene_first_line > cur_last_line:
            # only add scene if last_line > first_line and first_line > cur_last_line
            formatted_chapter_scenes.append(formatted_scene)
            formatted_chapter = cur_chapter.replace("\n", " ")
            # replace weird quotes too
            formatted_chapter = formatted_chapter.replace("\"", " ").replace("'", " ")
            cur_last_line = scene_last_line

    # sort scenes by first_line
    formatted_chapter_scenes = sorted(formatted_chapter_scenes, key=lambda x: x["first_line"])

    # split chapter into scenes based on formatted_chapter_scenes
    scenes = []
    cur_min_index = 0
    for i, scene in enumerate(formatted_chapter_scenes):
        cur_first_line = f"LINE {scene['first_line']}:"
        # cur_first_line = formatted_chapter_scenes[i]["first_line"].strip("...").replace("\"", " ").replace("'", " ")
        first_line_index = formatted_chapter.find(cur_first_line, cur_min_index)
        if first_line_index == -1:
            # see if first_line_index overlaps with last_line of previous scene
            other_possible_first_index = formatted_chapter.find(cur_first_line)
            if other_possible_first_index != -1:
                overlap = cur_chapter[other_possible_first_index:cur_min_index]
                print(f"SCENE {i}")
                print("\nPREVIOUS SCENE")
                print(scenes[i-1])
                print("\nOVERLAP")
                print(overlap)
                print()
            first_line_index = cur_min_index
        cur_last_line = f"LINE {scene['last_line'] + 1}:"
        # cur_last_line = formatted_chapter_scenes[i]["last_line"].strip("...").replace("\"", " ").replace("'", " ")
        last_line_index = formatted_chapter.find(cur_last_line, first_line_index)
        # last_line_index += len(cur_last_line)
        if last_line_index == -1:
            last_line_index = len(cur_chapter)
        scene_text = cur_chapter[first_line_index:last_line_index].strip()
        # remove line numbers
        scene_text = re.sub(r'LINE \d+: ', '', scene_text)
        scenes.append(scene_text)
        cur_min_index = last_line_index

    # add scene text to formatted_chapter_scenes
    for i, scene in enumerate(formatted_chapter_scenes):
        formatted_chapter_scenes[i]["text"] = scenes[i]

    return scenes, formatted_chapter_scenes

In [1065]:
async def get_scene_details_async(scenes, formatted_chapter_scenes, chapter_num):
    print(f"Generating scene details for Chapter {markers[chapter_num]}...")
    all_scene_details = []

    async def process_scene(scene_text, cur_scene):
        """Async function to analyze a single scene"""
        if len(scene_text) == 0:
            return
        
        start_prompt = """
                          When analyzing this scene, focus on characters with a significant role in the scene.
                          If you can't determine the character's role or emotion, don't include them in the analysis.

                          Make sure to only use direct quotes from the text.
                          Don't paraphrase or change the text in any way.
                          If the character speaks, the quote should be something they say.
                          If the character is only described, the quote should be a description of the character's actions or emotions
                          (or any quote that mentions them, even if it is said by another character).
                          Try your best to find a direct quote for each character, but if you can't find a quote, 
                          (1) set the "quote" field to "No quote available" and (2) in the "fake_quote" field,
                          write a brief explanation of the character's emotions and actions in the scene.
                          Leave the "fake_quote" field empty if you can find a direct quote.
                        
                          Scene text:

                          """
        if analysis_type == "theme":
            start_prompt = """
                            When analyzing this scene, focus on themes with a significant role in the scene.
                            If you can't determine the theme's role or emotion, don't include it in the analysis.

                            Make sure to only use direct quotes from the text that describe the theme or relate to it.
                            Don't paraphrase or change the text in any way.
                            Try your best to find a direct quote for each theme, but if you can't find a quote,
                            (1) set the "quote" field to "No quote available" and (2) in the "fake_quote" field,
                            write a brief explanation of the theme in this scene.
                            Leave the "fake_quote" field empty if you can find a direct quote.
                        
                            Scene text:

                            """
            
        invalid_input = True
        while invalid_input:
            try:
                if analysis_type == "theme":
                    scene_details = await scene_details_llm_theme.ainvoke(start_prompt + cur_scene)
                else:
                    scene_details = await scene_details_llm.ainvoke(start_prompt + cur_scene)
                invalid_input = False
                # print(scene_details)
            except:
                invalid_input = True
        
        return scene_details

    # Run all scene details in parallel
    tasks = [process_scene(formatted_chapter_scenes[i]["text"], scenes[i]) for i in range(len(scenes))]
    all_scene_details = await asyncio.gather(*tasks)

    # add details to formatted_chapter_scenes
    for i, scene in enumerate(formatted_chapter_scenes):
        scene_text = formatted_chapter_scenes[i]["text"]
        text_length = len(scene_text)
        num_lines = formatted_chapter_scenes[i]["last_line"] - formatted_chapter_scenes[i]["first_line"] + 1
        # first_line = formatted_chapter_scenes[i]["first_line"]
        # first_line_length = len(first_line)
        # last_line = formatted_chapter_scenes[i]["last_line"]
        # last_line_length = len(last_line)

        # if text_length == 0 or text_length < first_line_length + last_line_length:
        if text_length == 0:
            # no scene text or scene text is too short
            continue
        
        cur_scene_details = all_scene_details[i]
        formatted_chapter_scenes[i]["length"] = text_length
        formatted_chapter_scenes[i]["num_lines"] = num_lines
        formatted_chapter_scenes[i]["location"] = cur_scene_details.location
        formatted_chapter_scenes[i]["importance"] = cur_scene_details.importance
        formatted_chapter_scenes[i]["conflict"] = cur_scene_details.conflict
        formatted_character_info = []

        characters = cur_scene_details.themes if analysis_type == "theme" else cur_scene_details.characters
        # if model_type == "gemini":
        #     characters = [DotDict(character) for character in characters]
        for character in characters:
            character_info = {
                "name": character.name,
                "role": character.role,
                "importance": character.importance,
                "emotion": character.emotion,
                "sentiment": character.sentiment,
                "quote": character.quote,
                "fake_quote": character.fake_quote
            }
            # add character only if role is not N/A and emotion is not N/A and quote is not "No quote available"
            if not (character.role == "N/A" and character.emotion == "N/A" and "No quote available" in character.quote):
                formatted_character_info.append(character_info)
        formatted_chapter_scenes[i]["characters"] = formatted_character_info

    return formatted_chapter_scenes

In [1316]:
async def process_all_chapters_async():
    # Step 1: Split each chapter into scenes in parallel
    chapter_tasks = [get_scene_info_async(chapters, i) for i in range(len(chapters))]
    chapter_results = await asyncio.gather(*chapter_tasks)

    # extract scenes and formatted_chapter_scenes
    scenes = [result[0] for result in chapter_results]
    formatted_chapter_scenes = [result[1] for result in chapter_results]

    # Step 2: Get scene details for each scene in parallel
    detail_tasks = [semaphore_wrapper(get_scene_details_async, scenes[i], formatted_chapter_scenes[i], i) for i in range(len(chapters))]
    final_formatted_scenes = await asyncio.gather(*detail_tasks)

    # Step 3: Filter and save results
    for i, marker in enumerate(markers):
        formatted_chapter_scenes = final_formatted_scenes[i]

        # remove scenes with no text
        formatted_chapter_scenes = [scene for scene in formatted_chapter_scenes if scene["text"] != "" and "characters" in scene]
        # remove characters with no quote
        for scene in formatted_chapter_scenes:
            scene["characters"] = [character for character in scene["characters"] if "(" not in character["quote"] != "" and "<" not in character["name"] and "unnamed" not in character["name"].lower()]
            # print(scene)
        
        # remove scenes with no characters
        formatted_chapter_scenes = [scene for scene in formatted_chapter_scenes if len(scene["characters"]) > 0]

        # save as json
        with open(f"json/{story_name}/{marker}.json", "w") as f:
            json.dump(formatted_chapter_scenes, f, indent=4)

        print(f"All details for Chapter {marker} saved.\n")

In [2506]:
# RUN MAIN SCENE LOOP ASYNC that will get all the scene/character/etc. info for each chapter
await process_all_chapters_async()

Splitting Chapter Chapter Two The Beginning into scenes...
Splitting Chapter Chapter Three Days into scenes...
Splitting Chapter Chapter Four The Search into scenes...
Splitting Chapter Chapter Five Memories into scenes...
Splitting Chapter Chapter Six Reenactment into scenes...
Splitting Chapter Chapter Seven Struggle, Magnificently into scenes...
Splitting Chapter Chapter Eight Your Name. into scenes...
FIRST LINE: 3
LAST LINE: 14

FIRST LINE: 20
LAST LINE: 34

FIRST LINE: 35
LAST LINE: 46

FIRST LINE: 47
LAST LINE: 49

FIRST LINE: 52
LAST LINE: 59

FIRST LINE: 60
LAST LINE: 66

FIRST LINE: 70
LAST LINE: 86

FIRST LINE: 3
LAST LINE: 28

FIRST LINE: 29
LAST LINE: 57

FIRST LINE: 58
LAST LINE: 70

FIRST LINE: 85
LAST LINE: 175

FIRST LINE: 188
LAST LINE: 265

FIRST LINE: 266
LAST LINE: 284

FIRST LINE: 309
LAST LINE: 351

FIRST LINE: 3
LAST LINE: 26

FIRST LINE: 41
LAST LINE: 66

FIRST LINE: 90
LAST LINE: 108

FIRST LINE: 110
LAST LINE: 240

FIRST LINE: 241
LAST LINE: 368

FIRST LINE: 

In [2507]:
# rank each scene by importance
# and within each scene, the characters

for i, marker in enumerate(markers):
    print(marker)
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    importances = []
    conflicts = []
    
    # extract importance from each scene
    for j, scene in enumerate(scenes):
        importances.append((j, scene["importance"]))
        conflicts.append((j, scene["conflict"]))
        # now extract character importances
        character_importances = []
        for k, character in enumerate(scene["characters"]):
            character_importances.append((k, character["importance"]))
        # sort character importances
        sorted_character_importances = sorted(character_importances, key=lambda x: x[1], reverse=True)
        # add importance_rank to each character
        for k, (l, _) in enumerate(sorted_character_importances):
            scenes[j]["characters"][l]["importance_rank"] = k+1
        # add number of each scene
        scenes[j]["number"] = j+1
    # sort importances
    sorted_importances = sorted(importances, key=lambda x: x[1], reverse=True)
    # add importance_rank to each scene
    for k, (j, _) in enumerate(sorted_importances):
        scenes[j]["importance_rank"] = k+1
    # sort conflicts
    sorted_conflicts = sorted(conflicts, key=lambda x: x[1], reverse=True)
    # add conflict_rank to each scene
    for k, (j, _) in enumerate(sorted_conflicts):
        scenes[j]["conflict_rank"] = k+1
    
    # print results
    # for j, scene in enumerate(scenes):
    #     print(scenes[j]["title"], scenes[j]["importance_rank"])
    #     print("--------------------------------")
    #     for k, character in enumerate(scene["characters"]):
    #         print(character["name"], character["importance_rank"])
    #     print()

    # save as json
    with open(f"json/{story_name}/{marker}.json", "w") as f:
        json.dump(scenes, f, indent=4)

Chapter Two The Beginning
Chapter Three Days
Chapter Four The Search
Chapter Five Memories
Chapter Six Reenactment
Chapter Seven Struggle, Magnificently
Chapter Eight Your Name.


In [2508]:
# combine all chapters into one json file
all_scenes = []
for i, marker in enumerate(markers):
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    all_scenes += scenes

with open(f"json/{story_name}/all.json", "w") as f:
    json.dump(all_scenes, f, indent=4)

### grouping duplicate characters / location names

In [2509]:
# collect all unique characters and locations for each chapter
all_characters = {}
all_locations = []
for i, marker in enumerate(markers):
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    for scene in scenes:
        for character in scene["characters"]:
            char = character["name"]
            if char not in all_characters:
                all_characters[char] = character["role"]
        location = scene["location"]
        if location not in all_locations:
            all_locations.append(location)

In [2510]:
len(all_characters), len(all_locations)

(195, 79)

In [2511]:
all_characters

{'Loneliness': "Highlights the girl's emotional state",
 'Connection': 'Represents the desire for a bond',
 'Dreams': 'Symbolizes the blurred line between reality and imagination',
 'Self-Discovery': 'The protagonist realizes her identity as a girl.',
 'Body Awareness': 'The protagonist explores and reacts to her physical form.',
 'Innocence and Youth': 'The interaction with the younger sister highlights innocence.',
 'Family Dynamics': 'Explores the relationship between siblings and their interactions.',
 'Community Connection': 'Highlights the small-town atmosphere and shared experiences.',
 'Adult Responsibilities': 'Reflects on the burdens and complexities of adult issues.',
 'Celestial Events': 'Symbolizes hope and wonder amidst daily life.',
 'Friendship Dynamics': 'Highlights the playful banter and camaraderie among friends.',
 'Community Pressure': 'Illustrates the expectations and scrutiny from the community regarding family and social status.',
 'Nostalgia': 'Evokes a sense o

In [2512]:
all_locations

['On a train',
 'Bedroom in the morning',
 'Itomori Living Room',
 'Itomori Lake area',
 'Country high school classroom',
 'Schoolyard in Itomori',
 'Neighborhood bus stop',
 'Miyamizu Shrine workroom',
 'Miyamizu Shrine',
 'Shrine steps at night',
 'Strange Bathroom',
 'Living room',
 'Small messy room',
 'Japanese high school',
 'Rooftop at school',
 'A cafe after class',
 'Ritzy Italian restaurant',
 'Restaurant Back Room',
 'Tokyo train home',
 "Taki's workplace",
 'Classroom',
 "Character's Room",
 'Mountain Shrine Trail',
 'Near the great tree',
 'Tokyo, Roppongi',
 'Sketchbook and Room',
 'Tokyo Station',
 'On the train to Nagoya',
 'On the express train Hida',
 'Itomori High School grounds',
 'Next room',
 'New Itomori Lake',
 'Crater-shaped basin',
 'Village by the lake',
 'Hospital Room',
 'Family Shrine',
 "Mitsuha's House",
 "Mitsuha's memories",
 "Mitsuha's Grandmother's Room",
 'Night of the festival',
 "Mitsuha's body",
 "Sister's Room",
 'Tea table conversation',
 'Town

In [2513]:
# # create csv file for characters
# import csv

# with open(f"csv/{story_name}_characters.csv", "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerow(["Character", "Role"])
#     for char, role in all_characters.items():
#         writer.writerow([char, role])

In [2514]:
# # create csv file for locations
# with open(f"csv/{story_name}_locations.csv", "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerow(["Location"])
#     for location in all_locations:
#         writer.writerow([location])

In [2515]:
# Assuming your JSON file is stored at 'all.json'
with open(f"json/{story_name}/all.json") as f:
    json_data = json.load(f)

prompt = """
You are given a list of characters with some duplicates or alternate names.
Your task is to group all the alternate names for each unique character under a single full name, using the descriptions as guidance.
Don't keep characters like "Narrator" unless they are never referred to by another name.
Separate characters that are related to others (e.g., "Bob" and "Ms. Doe (Bob's mom)" and "Bob's sister") into separate entries.

First create a new character list by removing all duplicates and alternate names and only keeping the full name for each character.

Then, output a JSON dictionary where the key is each character in your new list and the value is a list of all possible alternate names. 
All names should be taken directly from the original character list and should only appear once in the output.

For example:
{{
    "Bob Smith": ["Bob", "Bobby", "Robert"],
    "Jane Doe": ["Jane", "Ms. Doe", "Bob's mother"]
}}
"""

theme_prompt = """
You are given a list of themes with some duplicates or alternate names.
Your task is to group all the alternate names for each unique theme under a single name, using the descriptions as guidance.
Your goal is to have as few themes as possible while still capturing all the different ways the themes are described.

First create a new theme list by removing all duplicates and alternate names and only keeping the best description for each theme.

Then, output a JSON dictionary where the key is each theme in your new list and the value is a list of all possible alternate names. 
All names should be taken directly from the original theme list and should only appear once in the output.

For example:
{{
    "Memory": ["Remembering the past", "Recollection", "Nostalgia"],
    "Friendship and Loyalty": ["Friendship and Companionship", "Making Friends", "Bonds of Friendship", "Loyalty and Friendship"]
}}
"""

location_prompt = """
You are given a list of locations with some duplicates or alternate names.
Your task is to group all the alternate or similiar names for each unique location under a single location name.

First create a new location list by removing all duplicates and alternate names and only keeping the full name for each location.
If one location is inside another or nearby to it, only keep the larger location (e.g., "Bob's bedroom", "Bob's garden", and "Bob Smith's castle" can be listed under "Bob's house").
Then, output a JSON dictionary where the key is each location in your new list and the value is a list of all possible alternate location names.
All locations should be taken directly from the original location list and should only appear once in the output dict.

For example:
{{
    "Los Angeles": ["LA", "City of Angels", "Cafe in Los Angeles"],
    "Bob's house": ["Bob's bedroom", "Bob Smith's garden", "Bob's castle"]
}}
"""

if analysis_type == "theme":
    prompt = theme_prompt

# if using llm2 (claude)
prompt += "Just output the JSON dictionary as the final result without any additional information."
location_prompt += "Just output the JSON dictionary as the final result without any additional information."

prompt_template = ChatPromptTemplate.from_messages([("system", prompt), ("human", "{input}")])
location_prompt_template = ChatPromptTemplate.from_messages([("system", location_prompt), ("human", "{input}")])

In [2516]:
# Create the LLM chain with the prompt template and model

# Run the chain by passing the characters and JSON data as inputs
if analysis_type == "theme":
    final_prompt = prompt_template.invoke(f"theme list: {all_characters}")
else:
    final_prompt = prompt_template.invoke(f"character list: {all_characters}")
# character_llm = llm.with_structured_output(None, method="json_mode") if model_type == "gpt" else llm
character_llm = llm2
# character_llm = llm
output = character_llm.invoke(final_prompt)


In [2517]:
output

AIMessage(content='{\n    "Loneliness": ["Isolation"],\n    "Connection": ["Connection to Nature", "Connection to Heritage", "Connection to Life", "Connection and Musubi", "Connection to Place"],\n    "Dreams": ["Dreams and Aspirations", "Dream vs Reality", "Dream vs. Reality"],\n    "Self-Discovery": ["Identity Crisis", "Search for Identity"],\n    "Body Awareness": ["Physical Transformation"],\n    "Innocence and Youth": ["Innocence of Youth", "Innocence"],\n    "Family Dynamics": ["Family and Duty", "Family Bonds", "Family Conflict", "Family Support"],\n    "Community Connection": ["Community Awareness"],\n    "Adult Responsibilities": ["Responsibility"],\n    "Celestial Events": ["Cosmic Wonder", "Cosmic Challenge"],\n    "Friendship": ["Friendship Dynamics", "Friendship and Camaraderie"],\n    "Community Pressure": ["Social Pressure", "Peer Pressure"],\n    "Nostalgia": ["Nostalgia for Language", "Nostalgia and Memory", "Nostalgia and Loss"],\n    "Political Ambition": [],\n    "T

In [2518]:
# if model_type == "claude":
output_formatted = output.content
# if output stop_reason = "max_tokens",cut off at last ']' character and add '}'
if output_formatted[-1] != "}":
    last_bracket_ind = output_formatted.rfind("]")
    output_formatted = output_formatted[:last_bracket_ind + 1] + "}"
    # print(output_formatted)
# turn into json
output = json.loads(output_formatted)

In [2519]:
output

{'Loneliness': ['Isolation'],
 'Connection': ['Connection to Nature',
  'Connection to Heritage',
  'Connection to Life',
  'Connection and Musubi',
  'Connection to Place'],
 'Dreams': ['Dreams and Aspirations', 'Dream vs Reality', 'Dream vs. Reality'],
 'Self-Discovery': ['Identity Crisis', 'Search for Identity'],
 'Body Awareness': ['Physical Transformation'],
 'Innocence and Youth': ['Innocence of Youth', 'Innocence'],
 'Family Dynamics': ['Family and Duty',
  'Family Bonds',
  'Family Conflict',
  'Family Support'],
 'Community Connection': ['Community Awareness'],
 'Adult Responsibilities': ['Responsibility'],
 'Celestial Events': ['Cosmic Wonder', 'Cosmic Challenge'],
 'Friendship': ['Friendship Dynamics', 'Friendship and Camaraderie'],
 'Community Pressure': ['Social Pressure', 'Peer Pressure'],
 'Nostalgia': ['Nostalgia for Language',
  'Nostalgia and Memory',
  'Nostalgia and Loss'],
 'Political Ambition': [],
 'Twilight Symbolism': [],
 'Memory': ['Memory and Forgetting',
  

In [2520]:
# Create the LLM chain with the prompt template and model

# Run the chain by passing the characters and JSON data as inputs
final_location_prompt = location_prompt_template.invoke(f"location list: {all_locations}")
# location_llm = llm.with_structured_output(None, method="json_mode") if model_type == "gpt" else llm
location_llm = llm2
# location_llm = llm
location_output = location_llm.invoke(final_location_prompt)

In [2521]:
location_output

AIMessage(content='{\n    "Train": ["On a train", "Tokyo train home", "On the train to Nagoya", "On the express train Hida"],\n    "Bedroom": ["Bedroom in the morning", "Strange Bathroom", "Small messy room", "Character\'s Room", "Next room"],\n    "Itomori": ["Itomori Living Room", "Itomori Lake area", "Schoolyard in Itomori", "Itomori High School grounds", "New Itomori Lake", "Crater-shaped basin", "Village by the lake", "Town of Itomori", "Itomori Lake at night", "Itomori Substation", "Itomori town", "Itomori Shrine Grounds", "Itomori High School", "Itomori Town Hall"],\n    "School": ["Country high school classroom", "Japanese high school", "Rooftop at school", "Classroom", "High school hill", "Abandoned club building", "School Broadcasting Room", "School during blackout"],\n    "Bus stop": ["Neighborhood bus stop"],\n    "Miyamizu Shrine": ["Miyamizu Shrine workroom", "Shrine steps at night", "Mountain Shrine Trail", "Family Shrine"],\n    "Living room": ["Living room", "Living ro

In [2522]:
location_output_formatted = location_output.content
# turn into json
location_output = json.loads(location_output_formatted)

In [2523]:
location_output

{'Train': ['On a train',
  'Tokyo train home',
  'On the train to Nagoya',
  'On the express train Hida'],
 'Bedroom': ['Bedroom in the morning',
  'Strange Bathroom',
  'Small messy room',
  "Character's Room",
  'Next room'],
 'Itomori': ['Itomori Living Room',
  'Itomori Lake area',
  'Schoolyard in Itomori',
  'Itomori High School grounds',
  'New Itomori Lake',
  'Crater-shaped basin',
  'Village by the lake',
  'Town of Itomori',
  'Itomori Lake at night',
  'Itomori Substation',
  'Itomori town',
  'Itomori Shrine Grounds',
  'Itomori High School',
  'Itomori Town Hall'],
 'School': ['Country high school classroom',
  'Japanese high school',
  'Rooftop at school',
  'Classroom',
  'High school hill',
  'Abandoned club building',
  'School Broadcasting Room',
  'School during blackout'],
 'Bus stop': ['Neighborhood bus stop'],
 'Miyamizu Shrine': ['Miyamizu Shrine workroom',
  'Shrine steps at night',
  'Mountain Shrine Trail',
  'Family Shrine'],
 'Living room': ['Living room', 

In [2524]:
# Function to find the correct key based on a search string, ensuring whole-word matches
def find_key_from_alias(text, character_dict):
    for key, aliases in character_dict.items():
        for alias in sorted(aliases, key=len, reverse=True):
            # Match the alias as a whole word, and ensure no partial matches within other words
            pattern = rf'\b{re.escape(alias)}\b(?!\'s)'  # (?!\'s) ensures no match for possessives like Mitsuha's
            if re.search(pattern, text):
                return key  # Return the key if the alias is found as a whole word
    return text  # Return the original text if no match is found

In [2525]:
# Load the scenes
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)

# Capitalize output and location output
capitalized_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in output.items()}
capitalized_location_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in location_output.items()}

# Process each scene
for scene in scenes:
    # Process characters
    for character in scene["characters"]:
        char = string.capwords(character["name"])
        found_char = False

        # Check if the character's name is a key in the dictionary
        if char in capitalized_output:
            found_char = True
            character["name"] = char  # Capitalize the character name
        
        # If not found in the dictionary, check the aliases
        if not found_char:
            # Find the right key for the alias (if exists)
            matched_key = find_key_from_alias(char, capitalized_output)
            if matched_key != char:
                character["name"] = matched_key
                found_char = True
        if not found_char:
            character["name"] = char

    # Process location
    loc = string.capwords(scene["location"])
    found_loc = False

    # Check if the location is a key in the location dictionary
    if loc in capitalized_location_output:
        found_loc = True
        scene["location"] = loc  # Capitalize the location

    # If not found in the dictionary, check the aliases
    if not found_loc:
        matched_key = find_key_from_alias(loc, capitalized_location_output)
        if matched_key != loc:
            scene["location"] = matched_key
            found_loc = True
    if not found_loc:
        scene["location"] = loc

# Save the updated scenes
with open(f"json/{story_name}/all.json", "w") as f:
    json.dump(scenes, f, indent=4)

In [2526]:
# now list all characters and locations again
# collect all unique characters and locations for each chapter
all_characters_new = {}
all_locations_new = []
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)
for scene in scenes:
    for character in scene["characters"]:
        char = character["name"]
        if char not in all_characters_new:
            all_characters_new[char] = 1
        else:
            all_characters_new[char] += 1
    location = scene["location"]
    if location not in all_locations_new:
        all_locations_new.append(location)

In [2527]:
len(all_characters_new), len(all_locations_new)

(118, 30)

In [2528]:
all_characters_new 

{'Loneliness': 12,
 'Connection': 10,
 'Dreams': 4,
 'Self-discovery': 7,
 'Body Awareness': 2,
 'Innocence And Youth': 3,
 'Family Dynamics': 7,
 'Community Connection': 2,
 'Adult Responsibilities': 4,
 'Celestial Events': 3,
 'Friendship': 11,
 'Community Pressure': 4,
 'Nostalgia': 10,
 'Political Ambition': 1,
 'Twilight Symbolism': 1,
 'Memory': 7,
 'Desire For Escape': 3,
 'Disappointment': 1,
 'Tradition And Heritage': 3,
 'Youth And Growth': 2,
 'Loss': 2,
 'Embarrassment': 1,
 'Peer Judgment': 2,
 'Generational Gap': 1,
 'Disorientation': 2,
 'Authority': 3,
 'Regret': 1,
 'Imagination And Creativity': 2,
 'Urgency': 5,
 'Social Observation': 1,
 'Contrast': 1,
 'Identity': 7,
 'Cultural Experience': 1,
 'Excitement': 1,
 'Inexperience': 1,
 'Blame And Frustration': 1,
 'Admiration': 1,
 'Support': 1,
 'Vulnerability': 2,
 'Urban Life': 1,
 'Romantic Interest': 2,
 'Social Dynamics': 1,
 'Spirituality': 1,
 'Ritual And Offering': 1,
 'Self-doubt And Insecurity': 2,
 'Daily Ro

In [2529]:

all_locations_new

['Train',
 'Bedroom',
 'Itomori',
 'School',
 'Bus Stop',
 'Miyamizu Shrine',
 'Living Room',
 'Cafe',
 'Restaurant',
 'Workplace',
 'Mountain',
 'Tokyo',
 'Sketchbook And Room',
 'Hospital Room',
 "Mitsuha's House",
 'Festival',
 'Tea Table',
 'Town Hall',
 "Teshigawara's Room",
 'Desk',
 'Hilltop',
 'Hushed Town',
 "Mitsuha's Location",
 'Under The Night Sky',
 'Dark Deer Track',
 'On The Slope',
 'Miyamizu Shrine Road',
 'Train Platform',
 'Interview Room',
 'On The Way Home']

## produce chapter summaries

In [2530]:
# Pydantic
class ChapterSummary(BaseModel):
    """Summarize a chapter based on scenes."""
    summary: str = Field(description="A brief, 1-line summary of the chapter")

chapter_llm = llm.with_structured_output(ChapterSummary)

In [2531]:
# remove any markers without any scenes
markers_to_remove = []
for i, marker in enumerate(markers):
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    if len(scenes) == 0:
        markers_to_remove.append(marker)

for marker in markers_to_remove:
    markers.remove(marker)

In [2532]:
async def read_json_async(file_path):
    """Asynchronously read a JSON file."""
    async with aiofiles.open(file_path, mode="r") as f:
        data = await f.read()
        return json.loads(data)
    
async def read_text_async(file_path):
    """Asynchronously read a text file."""
    async with aiofiles.open(file_path, mode="r") as f:
        return await f.read()
    
async def process_chapter_summary_async(marker):
    """Process the summary for a single chapter."""
    scenes_file = f"json/{story_name}/{marker}.json"
    chapter_file = f"chapters/{og_story_name}/{marker}.txt"

    # Read JSON and text files asynchronously
    scenes = await asyncio.gather(read_json_async(scenes_file), read_text_async(chapter_file))

    # Create scene summaries string
    scene_summaries = " ".join([scene["summary"] for scene in scenes[0]])

    # Get summary from LLM
    summary = await chapter_llm.ainvoke(scene_summaries)

    chapter_summary = {
        "chapter": marker,
        "scenes": len(scenes),
        "length": len(chapter),
        "num_lines": len(chapter.split("\n")),
        "summary": summary.summary,
        "importance": [],
        "conflict": [],
        "locations": {},
        "characters": {}
    }

    return chapter_summary

async def generate_chapter_summaries_async():
    """Generate summaries for all chapters in parallel."""
    # Processs chapters in parallel
    chapter_summaries = await asyncio.gather(*[process_chapter_summary_async(marker) for marker in markers])

    # Read all.json file asynchronously
    scenes = await read_json_async(f"json/{story_name}/all.json")

    # Aggregate character and location information
    for scene in scenes:
        for scene in scenes:
            chapter = scene["chapter"]
            chapter_summary = chapter_summaries[markers.index(chapter)]
            # add location to chapter summary
            location = scene["location"]
            chapter_summary_locations = chapter_summary["locations"]
            if location not in chapter_summary_locations:
                chapter_summary_locations[location] = 1
            else:
                chapter_summary_locations[location] += 1
            # add importance to chapter summary
            importance = scene["importance"]
            chapter_summary["importance"].append(importance)
            # add conflict to chapter summary
            conflict = scene["conflict"]
            chapter_summary["conflict"].append(conflict)
            # add characters to chapter summary
            characters = scene["characters"]
            for character in characters:
                char = character["name"]
                chapter_summary_characters = chapter_summary["characters"]
                if char not in chapter_summary_characters:
                    chapter_summary_characters[char] = 1
                else:
                    chapter_summary_characters[char] += 1

    # now average importance and conflict for each chapter
    for chapter_summary in chapter_summaries:
        if len(chapter_summary["importance"]) == 0:
            print(f"Chapter {chapter_summary['chapter']} has no scenes.")
            continue
        chapter_summary["importance"] = round(sum(chapter_summary["importance"]) / len(chapter_summary["importance"]), 2)
        chapter_summary["conflict"] = round(sum(chapter_summary["conflict"]) / len(chapter_summary["conflict"]), 2)

    print("Importances", [chapter_summary["importance"] for chapter_summary in chapter_summaries])
    print("Conflicts", [chapter_summary["conflict"] for chapter_summary in chapter_summaries])

    # rank each chapter by importance and conflict
    importances = []
    conflicts = []
    for chapter_summary in chapter_summaries:
        importances.append((chapter_summary["chapter"], chapter_summary["importance"]))
        conflicts.append((chapter_summary["chapter"], chapter_summary["conflict"]))
        
    # sort importances
    sorted_importances = sorted(importances, key=lambda x: x[1], reverse=True)
    # add importance_rank to each chaptera
    for k, (j, _) in enumerate(sorted_importances):
        chapter_summaries[markers.index(j)]["importance_rank"] = k+1

    # sort conflicts
    sorted_conflicts = sorted(conflicts, key=lambda x: x[1], reverse=True)
    # add conflict_rank to each chapter
    for k, (j, _) in enumerate(sorted_conflicts):
        chapter_summaries[markers.index(j)]["conflict_rank"] = k+1

    return chapter_summaries

In [2533]:
# create summary json of each chapter
chapter_summaries = await generate_chapter_summaries_async()

Importances [0.81, 0.83, 0.86, 0.84, 0.83, 0.87, 0.81]
Conflicts [0.57, 0.47, 0.6, 0.56, 0.49, 0.67, 0.46]


In [2534]:
chapter_summaries

[{'chapter': 'Chapter Two The Beginning',
  'scenes': 2,
  'length': 25027,
  'num_lines': 352,
  'summary': 'Taki navigates a day filled with confusion and memories of Mitsuha while preparing for the harvest festival with family and friends.',
  'importance': 0.81,
  'conflict': 0.57,
  'locations': {'Train': 82,
   'Bedroom': 82,
   'Itomori': 246,
   'School': 82,
   'Bus Stop': 82,
   'Miyamizu Shrine': 246},
  'characters': {'Loneliness': 164,
   'Connection': 82,
   'Dreams': 82,
   'Self-discovery': 164,
   'Body Awareness': 82,
   'Innocence And Youth': 164,
   'Family Dynamics': 164,
   'Community Connection': 82,
   'Adult Responsibilities': 82,
   'Celestial Events': 82,
   'Friendship': 246,
   'Community Pressure': 164,
   'Nostalgia': 246,
   'Political Ambition': 82,
   'Twilight Symbolism': 82,
   'Memory': 164,
   'Desire For Escape': 164,
   'Disappointment': 82,
   'Tradition And Heritage': 164,
   'Youth And Growth': 82,
   'Loss': 82,
   'Embarrassment': 82,
   'Pe

In [2535]:
# read in scenes from all_json
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)

In [2536]:
all_json = {}
all_json["title"] = story_info["title"]
all_json["type"] = story_info["type"]
if (all_json["type"] == "Movie"):
    all_json["director"] = story_info["director"]
else:
    all_json["author"] = story_info["author"]
all_json["year"] = story_info["year"]
if "url" in story_info:
    all_json["url"] = story_info["url"]
if "image" in story_info:
    all_json["image"] = story_info["image"]
all_json["num_chapters"] = len(chapters)
all_json["num_scenes"] = len(scenes)
all_json["num_characters"] = len(all_characters_new)
all_json["num_locations"] = len(all_locations_new)
all_json["chapters"] = chapter_summaries
all_json["scenes"] = scenes

## produce character summaries

In [2537]:
# Pydantic
class CharacterInfo(BaseModel):
    """Get information about a character in the story."""
    name: str = Field(description="Name of character.")
    quote: str = Field(description="Direct quote from the story that the character says or that describes them.")
    group: str = Field(description="Group that this character belongs to (e.g., main characters, upper class, talking animals). There should be at least 2 different groups of characters.")
    # color: str = Field(description="Unique aesthetic RGB color string that represents this character (e.g., rgb(118, 185, 71)). Every character should have a different color. Don't use white and make sure the color is visible against a white background.")
    # explanation: str = Field(description="Explanation why this color was chosen for this character.")

class LegendThemeInfo(BaseModel):
    """Get information about a theme in the story."""
    name: str = Field(description="Name of theme.")
    quote: str = Field(description="Direct quote from the story that describes this theme.")
    group: Optional[str] = Field(description="Group that this theme belongs to (e.g., Family, Social Themes, Emotions). There should be at least 2 different groups of themes. Make sure all groups are distinct and don't overlap with each other.")

class CharacterList(BaseModel):
    """Get overall information about the story."""
    # characterlist: list[CharacterInfo] = Field(description="List of character details. Make sure there is exactly one entry per character in the provided list and no additional characters are added. Choose a different color for each character.")
    characterlist: list[CharacterInfo] = Field(description="List of character details. Make sure there is exactly one entry per character in the provided list and no additional characters are added.")

class LegendThemeList(BaseModel):
     """Get overall information about the story."""
     themelist: list[LegendThemeInfo] = Field(description="List of theme details. Make sure there is exactly one entry per theme in the provided list and no additional themes are added.")

class ColorInfo(BaseModel):
    """Pick a color + explain why for each character"""
    name: str = Field(description="Name of character.")
    color: str = Field(description="Unique aesthetic RGB color string that represents this character (e.g., rgb(118, 185, 71)). Every character should have a different color. Don't use white and make sure the color is visible against a white background.")
    explanation: str = Field(description="Explanation why this color was chosen for this character.")

class ThemeColorInfo(BaseModel):
    """Pick a color + explain why for each theme"""
    name: str = Field(description="Name of theme.")
    color: str = Field(description="Unique aesthetic RGB color string that represents this theme (e.g., rgb(118, 185, 71)). Every theme should have a different color. Don't use white and make sure the color is visible against a white background.")
    explanation: str = Field(description="Explanation why this color was chosen for this theme.")

class ColorList(BaseModel):
    """List of color info for characters"""
    colorlist: list[ColorInfo] = Field(description="List of color details for each character. Make sure there is exactly one entry per character in the provided list and no additional characters are added. Choose a different color for each character.")

class ThemeColorList(BaseModel):
    """List of color info for themes"""
    colorlist: list[ThemeColorInfo] = Field(description="List of color details for each theme. Make sure there is exactly one entry per theme in the provided list and no additional themes are added. Choose a different color for each theme.")

if analysis_type == "theme":
    characterlist_llm = llm.with_structured_output(LegendThemeList)
    colorlist_llm = llm.with_structured_output(ThemeColorList)
else:
    characterlist_llm = llm.with_structured_output(CharacterList)
    colorlist_llm = llm.with_structured_output(ColorList)

In [2538]:
# sort all_characters_new by count, descending
sorted_characters = dict(sorted(all_characters_new.items(), key=lambda item: item[1], reverse=True))
# take top 20 characters
top_characters = dict(list(sorted_characters.items())[:20])
top_characters

{'Loneliness': 12,
 'Friendship': 11,
 'Connection': 10,
 'Nostalgia': 10,
 'Self-discovery': 7,
 'Family Dynamics': 7,
 'Memory': 7,
 'Identity': 7,
 'Desperation': 7,
 'Urgency': 5,
 'Destruction And Renewal': 5,
 'Fear': 5,
 'Dreams': 4,
 'Adult Responsibilities': 4,
 'Community Pressure': 4,
 "Nature's Influence": 4,
 'Innocence And Youth': 3,
 'Celestial Events': 3,
 'Desire For Escape': 3,
 'Tradition And Heritage': 3}

In [2539]:
character_arr = []
cur_arr = []
max_chars = 20

char_names = list(all_characters_new.keys())
# char_names = list(top_characters.keys())
for i, c in enumerate(char_names):
    cur_arr.append(c)
    if len(cur_arr) == max_chars or i == len(char_names) - 1:
        character_arr.append(cur_arr)
        cur_arr = []

for arr in character_arr:
    print(len(arr))


20
20
20
20
20
18


In [2540]:
async def process_character_chunk(i, arr):
    """Process a single chunk of character assignment asynchronously."""
    invalid_input = True
    while invalid_input:
        try:
            # filter scenes to include only those with relevant characters
            new_scenes = [scene for scene in scenes if any(char["name"] in arr for char in scene["characters"])]

             # take middle 126000 characters of new_scenes
            new_scenes_str = json.dumps(new_scenes)
            new_scenes_str = new_scenes_str[:126000]

            # construct prompt
            prompt = f"""characters: {arr}
                         story info: {new_scenes_str}

                         your output should contain exactly the same {len(arr)} characters as in the original list.

                         remember to only use direct quotes from the text and not paraphrase or change the text in any way.
                         """

            # invoke model
            res = await characterlist_llm.ainvoke(prompt)

            print(f"Chunk {i} processed.")
            return res

        except:
            print("Exception thrown. Trying again.")
            print("Error:", e)
            await asyncio.sleep(1) # wait for 1 second before trying again

async def process_all_characters():
    """Process all characters in parallel."""
    tasks = [process_character_chunk(i, arr) for i, arr in enumerate(character_arr)]
    results = await asyncio.gather(*tasks)
    return results

In [2541]:
# Run the main function to process all characters
all_res = await process_all_characters()

Chunk 5 processed.
Chunk 3 processed.
Chunk 1 processed.
Chunk 4 processed.
Chunk 0 processed.
Chunk 2 processed.


In [2542]:
# combine characterlists into one
characters = []
for res in all_res:
    if analysis_type == "theme":
        print(len(res.themelist))
        characters += res.themelist
    else:
        print(len(res.characterlist))
        characters += res.characterlist

len(characters)

20
20
20
20
20
18


118

In [2543]:
character_names = [char.name for char in characters]
for name in character_names:
    if name not in char_names:
        print("not in orig list:", name)

need_to_add = []
for name in char_names:
    if name not in character_names:
        print("not in llm output:", name)
        need_to_add.append(name)

# find duplicate names
unique_names = []
duplicate_indices = []
for i, name in enumerate(character_names):
    if name not in unique_names:
        unique_names.append(name)
    else:
        print("duplicate:", name, i)
        duplicate_indices.append(i)

print("need_to_add", len(need_to_add))
print("duplicate_indices", len(duplicate_indices))


need_to_add 0
duplicate_indices 0


### group duplicate character groups

In [2544]:
# group similar groups

group_prompt = """
You are given a list of groups with some duplicates or alternate names.
Your task is to group all the alternate names for each unique group under a single name.
Your goal is to have as few groups as possible while still capturing all the unique groups.

First create a new group list by removing all duplicates and alternate names and only keeping the best description for each group.

Then, output a JSON dictionary where the key is each group in your new list and the value is a list of all possible alternate names. 
All names should be taken directly from the original group list and should only appear once in the output.

For example:
{{
    "Main Characters": ["Main Characters", "Protagonists", "Heroes"],
    "Elders": ["Elders", "Family Authorities"],
    "Exploration": ["Exploration", "Adventure", "Discovery"],
    "Relationships": ["Relationships", "Friendships"]
}}
"""

# if using llm2 (claude)
group_prompt += "Just output the JSON dictionary as the final result without any additional information."

group_prompt_template = ChatPromptTemplate.from_messages([("system", group_prompt), ("human", "{input}")])

In [2545]:
# get all unique groups
# groups = [c['group'] for c in characters]
groups = [c.group for c in characters]
unique_groups = list(set(groups))
len(unique_groups), unique_groups

(28,
 ['Disaster',
  'Desire',
  'Nature',
  'Destruction',
  'Motivational Themes',
  'Urban Life',
  'Emotional Themes',
  'Identity',
  'Conflict',
  'Social Themes',
  'Emotions',
  'Family',
  'Family Dynamics',
  'Existential Themes',
  'Connection',
  'Imagination',
  'Career',
  'Culture',
  'Cultural Themes',
  'Loss',
  'Cultural Experience',
  'Introspection',
  'Memory',
  'Daily Life',
  'Crisis Themes',
  'Social Observation',
  'Spiritual Themes',
  'Ethical Themes'])

In [2546]:
# Create the LLM chain with the prompt template and model

# Run the chain by passing the characters and JSON data as inputs
if analysis_type == "theme":
    final_prompt = group_prompt_template.invoke(f"list of groups for themes: {unique_groups}")
else:
    final_prompt = group_prompt_template.invoke(f"list of groups for characters: {unique_groups}")
group_llm = llm2
output = group_llm.invoke(final_prompt)

In [2547]:
output

AIMessage(content='{\n  "Disaster": ["Disaster", "Destruction", "Crisis Themes"],\n  "Desire": ["Desire"],\n  "Nature": ["Nature"],\n  "Emotional Themes": ["Emotional Themes", "Emotions"],\n  "Urban Life": ["Urban Life", "Daily Life"],\n  "Identity": ["Identity"],\n  "Conflict": ["Conflict"],\n  "Social Themes": ["Social Themes", "Social Observation"],\n  "Family": ["Family", "Family Dynamics"],\n  "Existential Themes": ["Existential Themes"],\n  "Connection": ["Connection"],\n  "Imagination": ["Imagination"],\n  "Career": ["Career"],\n  "Cultural Themes": ["Cultural Themes", "Culture", "Cultural Experience"],\n  "Loss": ["Loss"],\n  "Introspection": ["Introspection"],\n  "Memory": ["Memory"],\n  "Spiritual Themes": ["Spiritual Themes"],\n  "Ethical Themes": ["Ethical Themes"],\n  "Motivational Themes": ["Motivational Themes"]\n}', additional_kwargs={}, response_metadata={'id': 'msg_01YK4c253uTWEa6XNoJ1hVHr', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'end_turn', 'stop_seque

In [2548]:
output_formatted = output.content
# turn into json
output = json.loads(output_formatted)
len(output), output

(20,
 {'Disaster': ['Disaster', 'Destruction', 'Crisis Themes'],
  'Desire': ['Desire'],
  'Nature': ['Nature'],
  'Emotional Themes': ['Emotional Themes', 'Emotions'],
  'Urban Life': ['Urban Life', 'Daily Life'],
  'Identity': ['Identity'],
  'Conflict': ['Conflict'],
  'Social Themes': ['Social Themes', 'Social Observation'],
  'Family': ['Family', 'Family Dynamics'],
  'Existential Themes': ['Existential Themes'],
  'Connection': ['Connection'],
  'Imagination': ['Imagination'],
  'Career': ['Career'],
  'Cultural Themes': ['Cultural Themes', 'Culture', 'Cultural Experience'],
  'Loss': ['Loss'],
  'Introspection': ['Introspection'],
  'Memory': ['Memory'],
  'Spiritual Themes': ['Spiritual Themes'],
  'Ethical Themes': ['Ethical Themes'],
  'Motivational Themes': ['Motivational Themes']})

In [2549]:
# Capitalize output and location output
capitalized_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in output.items()}

# Process each scene
for c in characters:

    # Process group
    group = string.capwords(c.group)
    found_group = False

    # Check if the group is a key in the dictionary
    if group in capitalized_output:
        found_group = True
        c.group = group  # Capitalize the group

    # If not found in the dictionary, check the aliases
    if not found_group:
        matched_key = find_key_from_alias(group, capitalized_output)
        if matched_key != group:
            c.group = matched_key
            found_group = True
    if not found_group:
        c.group = group

### assign colors to characters

In [2550]:
# now add colors for top characters
top_char_names = list(top_characters.keys())

all_res_colors = []
invalid_input = True
while invalid_input: # try until valid input
    # see if exception is thrown
    # filter all_json to only include characters in arr
    new_scenes = []
    for scene in scenes:
        include = False
        for character in scene["characters"]:
            if character["name"] in top_char_names:
                include = True
                break
        if include:
            new_scenes.append(scene)

    # take middle 126000 characters of new_scenes
    new_scenes_str = json.dumps(new_scenes)
    new_scenes_str = new_scenes_str[:126000]

    try:
        prompt = f"characters: {top_char_names}\nstory info: {new_scenes_str}\nyour output should contain exactly the same {len(top_char_names)} characters as in the original list. Make sure each character is assigned a unique color."
        res = colorlist_llm.invoke(prompt)
        all_res_colors.append(res)
        invalid_input = False
    except Exception as e:
        print("Exception thrown. Trying again.")
        print("Error:", e)
        invalid_input = True

In [2551]:
# find duplicate colors
colorlist = []
for res in all_res_colors:
    colorlist += res.colorlist
    
unique_colors = []
duplicate_colors = []
for i, char in enumerate(colorlist):
    color = char.color
    if color not in unique_colors:
        unique_colors.append(color)
    else:
        print("duplicate color:", color, i)
        duplicate_colors.append(i)

In [2552]:
colorinfo = {}

for c in colorlist:
    colorinfo[c.name] = {
        "color": c.color,
        "explanation": c.explanation
    }

colorinfo

{'Loneliness': {'color': 'rgb(70, 130, 180)',
  'explanation': 'This shade of blue represents the deep emotional state of loneliness, evoking feelings of sadness and isolation.'},
 'Friendship': {'color': 'rgb(255, 165, 0)',
  'explanation': 'Orange symbolizes warmth and connection, reflecting the joy and comfort found in friendships.'},
 'Connection': {'color': 'rgb(34, 139, 34)',
  'explanation': 'Green represents growth and harmony, embodying the desire for bonds and relationships.'},
 'Nostalgia': {'color': 'rgb(255, 192, 203)',
  'explanation': 'A soft pink evokes a sense of warmth and longing for the past, capturing the essence of nostalgia.'},
 'Self-discovery': {'color': 'rgb(148, 0, 211)',
  'explanation': 'Purple symbolizes creativity and self-reflection, representing the journey of self-discovery.'},
 'Family Dynamics': {'color': 'rgb(255, 215, 0)',
  'explanation': 'Gold reflects the value and complexity of family relationships, highlighting their importance.'},
 'Memory': 

In [2553]:
# remove duplicates
characters = [char for i, char in enumerate(characters) if i not in duplicate_indices]
print(len(characters))

118


In [2554]:
character_info = []
for c in characters:

    # see if character is in color_info
    name = c.name
    color = ""
    explanation = ""
    if name in colorinfo:
        color = colorinfo[name]["color"]
        explanation = colorinfo[name]["explanation"]
    
    character_info.append({
        "name": name,
        "quote": c.quote,
        "group": c.group,
        "color": color,
        "explanation": explanation
    })

len(character_info), character_info

(118,
 [{'name': 'Loneliness',
   'quote': "Her voice is earnest, pleading, as if she's about to cry. A voice trembling with loneliness, like the glimmer of distant stars.",
   'group': 'Emotional Themes',
   'color': 'rgb(70, 130, 180)',
   'explanation': 'This shade of blue represents the deep emotional state of loneliness, evoking feelings of sadness and isolation.'},
  {'name': 'Connection',
   'quote': 'My name is Mitsuha!',
   'group': 'Emotional Themes',
   'color': 'rgb(34, 139, 34)',
   'explanation': 'Green represents growth and harmony, embodying the desire for bonds and relationships.'},
  {'name': 'Dreams',
   'quote': "Still, it was just a dream. It doesn't mean anything.",
   'group': 'Imagination',
   'color': 'rgb(135, 206, 250)',
   'explanation': 'Light sky blue evokes a sense of calm and imagination, representing the realm of dreams.'},
  {'name': 'Self-discovery',
   'quote': "I'm...a girl?",
   'group': 'Identity',
   'color': 'rgb(148, 0, 211)',
   'explanation':

In [2555]:
for name in need_to_add:
    character_info.append({
        "name": name,
        "quote": "",
        "group": "misc",
        "color": "",
        "explanation": ""
    })

len(character_info), character_info

(118,
 [{'name': 'Loneliness',
   'quote': "Her voice is earnest, pleading, as if she's about to cry. A voice trembling with loneliness, like the glimmer of distant stars.",
   'group': 'Emotional Themes',
   'color': 'rgb(70, 130, 180)',
   'explanation': 'This shade of blue represents the deep emotional state of loneliness, evoking feelings of sadness and isolation.'},
  {'name': 'Connection',
   'quote': 'My name is Mitsuha!',
   'group': 'Emotional Themes',
   'color': 'rgb(34, 139, 34)',
   'explanation': 'Green represents growth and harmony, embodying the desire for bonds and relationships.'},
  {'name': 'Dreams',
   'quote': "Still, it was just a dream. It doesn't mean anything.",
   'group': 'Imagination',
   'color': 'rgb(135, 206, 250)',
   'explanation': 'Light sky blue evokes a sense of calm and imagination, representing the realm of dreams.'},
  {'name': 'Self-discovery',
   'quote': "I'm...a girl?",
   'group': 'Identity',
   'color': 'rgb(148, 0, 211)',
   'explanation':

## produce location summaries

In [2556]:
# Pydantic
class LocationInfo(BaseModel):
    """Get information about a location in the story."""
    name: str = Field(description="Name of location.")
    quote: str = Field(description="Direct quote from the story that describes this location")
    emoji: str = Field(description="Emoji that represents this location (e.g., 🏰). Make sure each location has a different emoji.")

class LocationList(BaseModel):
    """Get overall information about the story."""
    locationList: list[LocationInfo] = Field(description="List of location details. One entry per location.")

locationList_llm = llm.with_structured_output(LocationList)

In [2557]:
cropped_json = json.dumps(all_json)[:126000]
location_prompt = f"locations: {all_locations_new}\nstory info: {cropped_json}\nyour output should contain exactly the same {len(all_locations_new)} locations as in the original list."
invalid_input = True
while invalid_input: # try until valid input
    # see if exception is thrown
    try:
        res = locationList_llm.invoke(location_prompt)
        invalid_input = False
    except Exception as e:
        print("Exception thrown. Trying again.")
        print("Error:", e)
        invalid_input = True

In [2558]:
location_info = []
for l in res.locationList:
    location_info.append({
        "name": l.name,
        "quote": l.quote,
        "emoji": l.emoji
    })

print(len(location_info))
print(location_info)

30
[{'name': 'Train', 'quote': "The moment I realize this, I'm standing in a packed train car.", 'emoji': '🚆'}, {'name': 'Bedroom', 'quote': "I'm...a girl?", 'emoji': '🛏️'}, {'name': 'Itomori', 'quote': 'This place, Itomori, is a dinky little town with a population of fifteen hundred.', 'emoji': '🏞️'}, {'name': 'School', 'quote': "I watch the classroom from where I'm standing, half-hidden behind the door, timing my entrance.", 'emoji': '🏫'}, {'name': 'Bus Stop', 'quote': "The 'cafe' wasn't exactly what springs to mind.", 'emoji': '🚌'}, {'name': 'Miyamizu Shrine', 'quote': "Our braided cords hold a thousand years of Itomori's history.", 'emoji': '⛩️'}, {'name': 'Living Room', 'quote': 'Breakfast was your job today, kid. Remember? You overslept.', 'emoji': '🏠'}, {'name': 'Cafe', 'quote': "My mood is soaring, and I can't repress my excitement.", 'emoji': '☕'}, {'name': 'Restaurant', 'quote': "I'm being swept this way and that by a muddy torrent of confusion.", 'emoji': '🍽️'}, {'name': 'Wo

## generating the final json file!

In [2559]:
# add character and location info to all_json
all_json["characters"] = character_info
all_json["locations"] = location_info
all_json

{'title': 'Your Name',
 'type': 'Movie',
 'director': 'Makoto Shinkai',
 'year': '2016',
 'url': 'https://archive.org/stream/manga_Kimi_no_Na_wa/Kimi%20no%20Na%20wa%20%5BYen%20Press%5D_djvu.txt',
 'image': 'https://m.media-amazon.com/images/M/MV5BODRmZDVmNzUtZDA4ZC00NjhkLWI2M2UtN2M0ZDIzNDcxYThjL2ltYWdlXkEyXkFqcGdeQXVyNTk0MzMzODA@._V1_FMjpg_UX1000_.jpg',
 'num_chapters': 7,
 'num_scenes': 82,
 'num_characters': 118,
 'num_locations': 30,
 'chapters': [{'chapter': 'Chapter Two The Beginning',
   'scenes': 2,
   'length': 25027,
   'num_lines': 352,
   'summary': 'Taki navigates a day filled with confusion and memories of Mitsuha while preparing for the harvest festival with family and friends.',
   'importance': 0.81,
   'conflict': 0.57,
   'locations': {'Train': 82,
    'Bedroom': 82,
    'Itomori': 246,
    'School': 82,
    'Bus Stop': 82,
    'Miyamizu Shrine': 246},
   'characters': {'Loneliness': 164,
    'Connection': 82,
    'Dreams': 82,
    'Self-discovery': 164,
    'Body Awa

In [2560]:
# save as json
with open(f"json/{story_name}/final_data.json", "w") as f:
    json.dump(all_json, f, indent=4)